In [46]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
import json
from nltk.stem import PorterStemmer
from sklearn import linear_model


In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [3]:
users_reviews = []
for l in parse("review-Hawaii_10.json.gz"):
  users_reviews.append(l)

In [4]:
users_reviews[0]

{'user_id': '113965417079576625433',
 'name': 'manuel grimaldo',
 'time': 1591839903487,
 'rating': 5,
 'text': 'Great new upgrade',
 'pics': None,
 'resp': None,
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e'}

In [5]:
businesses = []
for l in parse("meta-Hawaii.json.gz"):
  businesses.append(l)

In [6]:
businesses[0]

{'name': 'Hale Pops',
 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762',
 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33',
 'description': None,
 'latitude': 21.637795699999998,
 'longitude': -157.9207142,
 'category': ['Restaurant'],
 'avg_rating': 4.4,
 'num_of_reviews': 18,
 'price': None,
 'hours': [['Thursday', '11AM–8PM'],
  ['Friday', '11AM–8PM'],
  ['Saturday', '11AM–8PM'],
  ['Sunday', 'Closed'],
  ['Monday', '11AM–8PM'],
  ['Tuesday', '11AM–8PM'],
  ['Wednesday', '11AM–8PM']],
 'MISC': {'Service options': ['Outdoor seating', 'Takeout', 'Delivery'],
  'Popular for': ['Lunch', 'Solo dining'],
  'Accessibility': ['Wheelchair accessible entrance'],
  'Offerings': ['Comfort food', 'Quick bite'],
  'Amenities': ['Good for kids'],
  'Atmosphere': ['Casual'],
  'Crowd': ['Groups', 'Tourists'],
  'Payments': ['NFC mobile payments']},
 'state': 'Closed ⋅ Opens 11AM',
 'relative_results': ['0x7c00451360f80cf1:0x930291a38bab3132',
  '0x7c00457322571d57:0xe3974e89bbfc41ce',

## Linear Regression

In [8]:
#user avg rating
user_ratings = {}

for review in users_reviews:
    user_id = review.get('user_id')
    rating = review.get('rating')

    if user_id is not None and rating is not None:
        if user_id not in user_ratings:
            user_ratings[user_id] = {'total_rating': 0, 'count': 0}
        
        user_ratings[user_id]['total_rating'] += rating
        user_ratings[user_id]['count'] += 1

user_avg_ratings = {user_id: data['total_rating'] / data['count'] 
                    for user_id, data in user_ratings.items() if data['count'] > 0}

for user_id in list(user_avg_ratings.keys())[:5]:
    print(f"User ID: {user_id}, Average Rating: {user_avg_ratings[user_id]:.2f}")

User ID: 113965417079576625433, Average Rating: 4.91
User ID: 116655819137293331166, Average Rating: 4.73
User ID: 100834119994550070853, Average Rating: 4.94
User ID: 103207214144482097315, Average Rating: 4.64
User ID: 108526171163172578599, Average Rating: 3.48


In [9]:
#businuess img count
image_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    images = review.get('pics')

    if gmap_id and images:
        num_images = len(images)
        if gmap_id in image_counts:
            image_counts[gmap_id] += num_images
        else:
            image_counts[gmap_id] = num_images

In [10]:
response_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    response = review.get('resp')

    if gmap_id and response:
        if gmap_id in response_counts:
            response_counts[gmap_id] += 1
        else:
            response_counts[gmap_id] = 1

In [11]:
users_data = [{'user_id': d['user_id'], 'gmap_id': d['gmap_id'], 'rating': d['rating'],'text': d['text']} 
              for d in users_reviews if 'user_id' in d and 'gmap_id' in d and 'rating' in d and 'text' in d]

business_dict = {d['gmap_id']: {'avg_rating': d.get('avg_rating', 0), 
                                'num_of_reviews': d.get('num_of_reviews', 0)}
                 for d in businesses if 'gmap_id' in d}

dataset = []
for user_review in users_data:
    gmap_id = user_review['gmap_id']
    if gmap_id in business_dict:
        merged_data = {**user_review, **business_dict[gmap_id]}
        dataset.append(merged_data)

In [12]:
#add features
for entry in dataset:
    gmap_id = entry.get('gmap_id')
    user_id = entry.get('user_id')
    entry['user_avg_rating'] = user_avg_ratings[user_id]
    entry['response_count'] = response_counts.get(gmap_id,0)
    entry['image_counts'] = image_counts.get(gmap_id, 0)
    

In [13]:
dataset[0]

{'user_id': '113965417079576625433',
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e',
 'rating': 5,
 'text': 'Great new upgrade',
 'avg_rating': 4.1,
 'num_of_reviews': 18,
 'user_avg_rating': 4.909090909090909,
 'response_count': 0,
 'image_counts': 0}

In [14]:
def MSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [15]:
# split dataset
train_data = dataset[:int(len(dataset) * 0.9)]
test_data = dataset[int(len(dataset) * 0.9):]
train_users_reviews = users_reviews[:int(len(users_reviews) * 0.9)]
test_users_reviews = users_reviews[int(len(users_reviews) * 0.9):]

### Baseline Model 1: avg_rating

In [16]:
# extract features
train_X = [[1, d['avg_rating']] for d in train_data]
test_X = [[1, d['avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [17]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [18]:
mse = MSE(predictions, test_y)
mse

0.6738557818215304

### Baseline Model 2: user_avg_rating

In [19]:
train_X = [[1, d['user_avg_rating']] for d in train_data]
test_X = [[1, d['user_avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [20]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [21]:
mse = MSE(predictions, test_y)
mse

0.5256570587970208

### Model 3: avg_rating, user_avg_rating

In [40]:
train_X = [[1, d['user_avg_rating'], d['avg_rating']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [41]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [42]:
mse = MSE(predictions, test_y)
mse

0.4886740944678856

### Model 4: avg_rating, user_avg_rating, num_of_reviews

In [25]:
train_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [26]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [27]:
mse = MSE(predictions, test_y)
mse

0.4894779561626342

### Model 5

In [32]:
train_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews'], d['image_counts']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews'], d['image_counts']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [33]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [34]:
mse = MSE(predictions, test_y)
mse

0.4891960794205272

## Bag of Words

In [43]:
# split dataset
train_data = dataset[:int(len(dataset) * 0.9)]
test_data = dataset[int(len(dataset) * 0.9):]
train_users_reviews = users_reviews[:int(len(users_reviews) * 0.9)]
test_users_reviews = users_reviews[int(len(users_reviews) * 0.9):]

In [48]:
# lowercase, punctuation removed, no stemming
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
# stemmer = PorterStemmer()
for d in train_data:
    if d['text']:
        r = ''.join([c for c in d['text'].lower() if not c in punctuation])
        for w in r.split():
            # w = stemmer.stem(w)
            wordCount[w] += 1
len(wordCount)

311792

In [49]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort(reverse=True)
words = [x[1] for x in counts[:1000]]

### Sentiment analysis

In [50]:
wordID = dict(zip(words, range(len(words))))
wordSet = set(words)

In [51]:
def feature(datum):
    feat = [0]*len(words)
    if d['text']:
        r = ''.join([c for c in d['text'].lower() if not c in punctuation])
        for w in r.split():
            if w in words:
                feat[wordID[w]] += 1
    feat.append(1) #offset
    return feat

In [52]:
train_X = [feature(d) for d in train_data]
train_y = [u['rating'] for u in train_users_reviews]
test_X = [feature(d) for d in test_data]
test_y = [u['rating'] for u in test_users_reviews]

In [ ]:
# regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(train_X, train_y)
train_predictions = clf.predict(train_X)
test_predictions = clf.predict(test_X)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
train_mse = MSE(train_predictions, train_y)
test_mse = MSE(test_predictions, test_y)
print(f'train mse: {train_mse}')
print(f'test mse: {test_mse}')